# `Mean Filter`
----

In [ ]:
import numpy as np
from numpy.typing import NDArray
from typing import TypeVar

F = TypeVar("F", float, np.floating)

def _mean_fil(image: NDArray[F], size: int = 3) -> NDArray[F]:
    """
    Applies a mean (box) filter to an image.

    Parameters:
    -----------
    image : NDArray[F]
        A 2D NumPy array representing the grayscale image.
    size : int, default=3
        The size of the mean filter kernel (must be an odd number).

    Returns:
    --------
    NDArray[F]
        The filtered image after applying the mean filter.

    Notes:
    ------
    - Uses a box filter (all ones kernel) of size `size × size`.
    - Computes the mean of each region using a convolution-like approach.
    - Uses zero-padding to maintain image dimensions.
    - Works only on grayscale images.
    """

    if size % 2 == 0 or size < 1:
        raise ValueError("Kernel size must be an odd positive integer.")

    # Get image dimensions
    image_y, image_x = image.shape

    # Compute padding size
    pad = size // 2

    # Apply zero-padding to the image
    padded_image = np.pad(image, pad_width = ((pad, pad), (pad, pad)), mode = 'constant', constant_values = 0)

    # Initialize output image
    filtered_image = np.zeros_like(image, dtype = np.float32)

    # Define kernel (uniform filter)
    kernel = np.ones((size, size), dtype = np.float32) / (size**2)

    # Perform mean filtering using vectorized slicing
    for y in range(size):
        for x in range(size):
            filtered_image += padded_image[y : y + image_y, x : x + image_x] * kernel[y, x]

    return filtered_image

if __name__ == "__main__":
    
    matrix = np.array([
        [10, 11, 9, 25, 22],
        [8, 10, 9, 26, 28],
        [9, 99, 9, 24, 25],
        [11, 11, 12, 23, 22],
        [10, 11, 9, 22, 25]
    ], dtype = np.float32)

    filtered_matrix = _mean_fil(matrix, size = 3)
    print("Filtered Image:\n", filtered_matrix)


# `Median Filter`
----

In [ ]:
import numpy as np
from numpy.typing import NDArray
from typing import TypeVar

F = TypeVar("F", float, np.floating)

def _median_fil(image: NDArray[F], kernel_size: int) -> NDArray[F]:
    """
    Applies a median filter to an image.

    Parameters:
    -----------
    image : NDArray[F]
        A 2D NumPy array representing the grayscale image.
    kernel_size : int
        The size of the median filter kernel (must be an odd number).

    Returns:
    --------
    NDArray[F]
        The filtered image after applying the median filter.

    Notes:
    ------
    - The median filter replaces each pixel with the **median** value of its surrounding pixels.
    - Works best for **removing salt-and-pepper noise**.
    - Uses **zero-padding** to maintain image dimensions.
    - Works only on **grayscale images**.
    """

    if kernel_size % 2 == 0 or kernel_size < 1:
        raise ValueError("Kernel size must be an odd positive integer.")

    # Get image dimensions
    image_y, image_x = image.shape

    # Compute padding size
    pad = kernel_size // 2

    # Apply zero-padding to the image
    padded_image = np.pad(image, pad_width = ((pad, pad), (pad, pad)), mode = 'constant', constant_values = 0)

    # Initialize output image
    filtered_image = np.zeros_like(image, dtype = np.float32)

    # Perform median filtering using a sliding window
    for y in range(image_y):
        for x in range(image_x):
            # Extract window
            window = padded_image[y : y + kernel_size, x : x + kernel_size]
            # Apply median filtering
            filtered_image[y, x] = np.median(window)

    return filtered_image


if __name__ == "__main__":
    matrix = np.array([
        [10, 11, 9, 25, 22],
        [8, 10, 9, 26, 28],
        [9, 99, 9, 24, 25],
        [11, 11, 12, 23, 22],
        [10, 11, 9, 22, 25]
    ], dtype = np.float32)

    filtered_matrix = _median_fil(matrix, kernel_size = 3)
    print("Filtered Image:\n", filtered_matrix)


# `Pixelize`
----

In [ ]:
import numpy as np
from numpy.typing import NDArray
from typing import TypeVar, Literal

I = TypeVar("I", bound = np.integer)

def pixelize(image: NDArray[I], bit: Literal[16, 32, 64] = 32) -> NDArray[I]:
    """
    Reduces the resolution of an image by averaging color values in square pixel blocks.

    This function divides an image into non-overlapping pixel blocks (kernels) and
    replaces each block with the average color of its pixels. The result is a pixelated
    version of the input image.

    Parameters:
    -----------
    image : np.ndarray
        The input image as a NumPy array with shape (height, width, channels).
        Must be a 3D array representing an RGB image.
    bit : int, optional
        The size of the pixel blocks in pixels (default: 32). Higher values increase
        the pixelation effect.

    Returns:
    --------
    np.ndarray
        The pixelized image as a NumPy array with the same shape as the input.

    Notes:
    ------
    - The function assumes the image has three color channels (RGB).
    - If the image dimensions are not exactly divisible by `bit`, the last pixels may
      be averaged over a smaller region.
    """

    image_y, image_x, _ = image.shape
    kernel_y, kernel_x = image_y // bit, image_x // bit
    pixelized_image = np.zeros_like(a = image, dtype = np.uint8)

    for y in range(0, image_y, kernel_y):
        for x in range(0, image_x, kernel_x):
            # kernel
            pixel_window = image[y : y + kernel_y, x : x + kernel_x] 
            pixel_color = pixel_window.mean(axis = (0, 1), dtype = int)
            # fill pixel with averaged color
            pixelized_image[y : y + kernel_y, x : x + kernel_x] = pixel_color
    
    return np.clip(pixelized_image, 0, 255).astype(dtype = np.int32)


if __name__ == "__main__":
    import cv2  # OpenCV for image handling
    import matplotlib.pyplot as plt

    # Load an image
    image = cv2.imread("pixelize/raccoon.jpg")  # Read as BGR
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB

    # Apply pixelization with a 32x32 block size
    pixelated_image = pixelize(image, bit = 32)

    # Display the result
    plt.imshow(pixelated_image)
    plt.axis("off")
    plt.show()